<a href="https://colab.research.google.com/github/VivianaWang/Repository-Test/blob/VivianaWang-patch-1/ST445_week5_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ST445 Managing and Visualizing Data
# More on web scraping and API

### Week 5 lab, MT 2020

# Before class

1. Week5 class attendance register (See Zoom chat for the link): 

    https://forms.office.com/Pages/ResponsePage.aspx?id=_epnVXfnpUKRu5RA_UO4k9IU9K3OWZRDnfHYZ0uayQlUMTFaRUtKTFJKV0NGUENPNk9ETEY2RTdEVC4u

2. A pull about if you have any questions regarding the week5's lecture material:
https://forms.gle/8kj8fauMdMfFNKZT8

# Plan for today

- More about web scraping (unicode and regular expression)
- Exercises

## Example: Web scraping the COPSS Awards Recipients

Get information regarding statisticians that were awarded with the COPSS Presidents' Award. Get a dataframe with 3 columns (`Year`, `Name`, `Institute`). For example (`1981`, `Peter J. Bickel`, `University of California, Berkeley`). Set `Year` to be the index, `Name` and `Institute` as column names.

https://en.wikipedia.org/wiki/COPSS_Presidents%27_Award

Step 1: Go to the website, use Google Chrome > More Tools > Developer Tools to inspect the HTML code. We well find all those informtion in `<li>...</li>`

Step 2: Find all these tags using `BeautifulSoup`. We want to select a subset of them.

Step 3: Use the `.split` function to separate the year, name and institute. Specify an optional argument k in `.split` to split only on the k-th occurrence.

In [ ]:
import requests
from bs4 import BeautifulSoup  
import pandas as pd
import re

In [ ]:
url='https://en.wikipedia.org/wiki/COPSS_Presidents%27_Award'
page=requests.get(url)
soup=BeautifulSoup(page.content,'lxml')

In [ ]:
lists = soup.find_all('li')
records = []
# print(lists)
for li in lists:
    # print(li.text)
    char_element = li.text.split(': ')
    if(len(char_element) == 2):
        # print(char_element)
        char = char_element[1].split(', ', 1)
        # print([char,len(char)])
        if(len(char) > 1):
            records.append([char_element[0],char[0],char[1]])
records
copss_recipients = pd.DataFrame(records, columns =['Year', 'Name', 'Institute'])
copss_recipients

## For each of these statisticians, find their year of birth. Create another column in your dataframe showing those information.

Step 1: Go to the website https://en.wikipedia.org/wiki/Peter_J._Bickel, use Google Chrome > More Tools > Developer Tools to inspect the HTML code.

Step 2: Find the tag containing the birth of year informtion and get the information using `BeautifulSoup`. We can use `soup.find(text='Born')` and `.findNext('td')`. To get birth of year from a give string, we can split it into two parts by 19 and get the first two characters in the second element. 

Step 3: To repeat this procedure for each of those statisticians in your dataframe, try something like this `'https://en.wikipedia.org/wiki/' + name`. It does not contain year of birth for all statisticians. If there are no such information for a particular statistician, you can set its value to be `NA`. 

In [ ]:
Birth = []
for name in copss_recipients['Name']:
    url = 'https://en.wikipedia.org/wiki/' + name.replace(' ', '_')
    # print(url)
    page=requests.get(url)
    soup=BeautifulSoup(page.content,'lxml')
    if(soup.find(text='Born') is not None):
        tmp = soup.find(text='Born').findNext('td').text.split('19')
        Birth.append('19' + tmp[1][:2])
    else:
        tmp = soup.find_all('p')[0].text
        tmp1 = re.search(r'born\s(\d{4})',tmp)
        if(tmp1 is not None):
            Birth.append(tmp1.group(1))
        else:
            Birth.append('NA')

copss_recipients['Birth year'] = Birth
copss_recipients

,Year,Name,Institute,Birth year
0,1981,Peter J. Bickel,"University of California, Berkeley",1940
1,1982,Stephen Fienberg,Carnegie Mellon University,1942
2,1983,Tze Leung Lai,Stanford University,1945
3,1984,David V. Hinkley,"University of California, Santa Barbara",NA
4,1985,James O. Berger,Duke University,1950
5,1986,Ross L. Prentice,Fred Hutchinson Cancer Research Center,1946
6,1987,C.F. Jeff Wu,Georgia Institute of Technology,1949
7,1988,Raymond J. Carroll,Texas A&M University,1949
8,1989,Peter Hall,Australian National University,NA
9,1990,Peter McCullagh,University of Chicago,1952


In [ ]:
url = 'https://en.wikipedia.org/wiki/C._F._Jeff_Wu'
page=requests.get(url)
soup=BeautifulSoup(page.content,'lxml')
tmp = re.search(r'born\s(\d{4})',soup.find_all('p')[0].text)
tmp.group(1)

'1949'

## Regular expression in python
- Wiki: https://en.wikipedia.org/wiki/Regular_expression
- Cheetsheet: https://www.debuggex.com/cheatsheet/regex/python
- Tutorial: https://developers.google.com/edu/python/regular-expressions, https://github.com/ziishaned/learn-regex
- Online practising https://regex101.com/

## Several examples
- Date format: YYYY-MM-DD r"\d{4}-\d{2}-\d{2}"
- text: My phone number is 421-2343-121, 
    - word with 6 characters r'\b\w{6}\b'
    - parse phone number (\d{3})-(\d{4})-(\d{3})

- test the strength of password

In [ ]:
text = '1981: Peter J. Bickel, University of California, Berkeley'
tmp = re.search(r'(\d{4}):\s+([^,]*),\s+(\w.*)', text)
tmp.group(1,2,3)

('1981', 'Peter J. Bickel', 'University of California, Berkeley')

## solution using regular expression


In [ ]:
lists = soup.find_all('li')
records = []
for li in lists:
    substr = re.search(r'(\d{4}):\s+([^,]*),\s+(\w.*)',li.text)
    if(substr is not None):
        records.append(substr.group(1,2,3))
copss_recipients = pd.DataFrame(records, columns =['Year', 'Name', 'Institute'])
copss_recipients

### Unicode 
A coding system that map every characters (letter from almost all languages, number, special characters, etc). 

- As of March 2020, it contains 143,859 characters
- In contrast, ASCII only contains 128 specified characters.
- Unicode standard:  UTF-8, UTF-16, etc
- wiki: https://en.wikipedia.org/wiki/Unicode
- https://unicode-table.com/en/
- En dash: '–' with unicode \u2013 in python

# Exercise: Web Scraping the Characters in the ``Three-Body" Novel Series From

## One implementation

In [ ]:
urls = ['https://en.wikipedia.org/wiki/The_Three-Body_Problem_(novel)',
        'https://en.wikipedia.org/wiki/The_Dark_Forest',
        'https://en.wikipedia.org/wiki/Death%27s_End']
characters = []
for i, url in enumerate(urls):
    page=requests.get(url)
    soup=BeautifulSoup(page.content,'lxml')
    lists = soup.find_all('li')
    for li in lists:
        element = re.split(' \u2013 ', li.text)
        if(len(element) < 2):
            continue
        characters.append([element[0], i])
characters

[['Ye Zhetai (叶哲泰)', 0],
 ['Shao Lin (绍琳)', 0],
 ['Ye Wenjie (叶文洁)', 0],
 ['Ye Wenxue (叶文雪)', 0],
 ['Lei Zhicheng (雷志成)', 0],
 ['Yang Weining (杨卫宁)', 0],
 ['Wang Miao (汪淼)', 0],
 ['Yang Dong (杨冬)', 0],
 ['Ding Yi (丁仪)', 0],
 ['Shi Qiang (史强)', 0],
 ['Chang Weisi (常伟思)', 0],
 ['Shen Yufei (申玉菲)', 0],
 ['Wei Cheng (魏成)', 0],
 ['Pan Han (潘寒)', 0],
 ['Sha Ruishan (沙瑞山)', 0],
 ['Mike Evans (麦克·伊文斯)', 0],
 ['Colonel Stanton (斯坦顿)', 0],
 ['Ye Wenjie (叶文洁)', 1],
 ['Mike Evans (麦克·伊文斯)', 1],
 ['Chang Weisi (常伟思)', 1],
 ['Zhang Beihai (章北海)', 1],
 ['Zhang Yuanchao (张援朝)', 1],
 ['Shi Qiang (史强), also nicknamed Da Shi (大史)', 1],
 ['Wu Yue (吴岳)', 1],
 ['Zhuang Yan (庄颜)', 1],
 ['Ding Yi (丁仪)', 1],
 ['Dongfang Yanxu (东方延绪)', 1],
 ['Kent (坎特)', 1],
 ['Frederick Tyler (弗雷德里克·泰勒)', 1],
 ['Manuel Rey Diaz (曼努尔·雷迪亚兹)', 1],
 ['Bill Hines (比尔·希恩斯)', 1],
 ['Luo Ji (罗辑)', 1],
 ['Cheng Xin (程心)', 2],
 ['Yun Tianming (云天明)', 2],
 ['Thomas Wade (托马斯·维德)', 2],
 ['Ai AA (艾AA)', 2],
 ['Luo Ji (罗辑)', 2],
 ['Guan Yif

## solution using regex

In [ ]:
# https://unicode-table.com/en/
text = 'Cheng Xin (程心) – Aerospace engineer from the early 21st century, second Swordholder'
re.match(r'(^\w.*)\s\u2013',text).group(1)

'Cheng Xin (程心)'

In [ ]:
urls = ['https://en.wikipedia.org/wiki/The_Three-Body_Problem_(novel)',
        'https://en.wikipedia.org/wiki/The_Dark_Forest',
        'https://en.wikipedia.org/wiki/Death%27s_End']
characters = []
for i, url in enumerate(urls):
    page=requests.get(url)
    soup=BeautifulSoup(page.content,'lxml')
    lists = soup.find_all('li')
    for li in lists:
        element = re.findall(r'(^\w.*)\s[–-]', li.text)
        # element = re.match(r'(^\w.*)\s–',text).group(1)
        if(len(element) > 0):
            characters.append([element[0], i+1])

characters

In [ ]:
characters_tab = pd.DataFrame(characters, columns=['Name', 'series'])
characters_tab

In [ ]:
characters_tab.groupby('Name').agg('count').sort_values('series', ascending=False)

# Exercise: using Openweathermap API to get current weather information in London

Hint 1: Go to https://openweathermap.org/appid (https://openweathermap.org/appid) to register an account, get your API key (https://home.openweathermap.org/api_keys).

Hint 2: To search the current weather for a particular city, try something like 'http://api.openweathermap.org/data/2.5/weather?q='+'city, country'+'&APPID='+api_key

Hint 3: Import json file. After you apply the get method, use .json to extract the weather information

You can get more weather report and forecast information here https://openweathermap.org/api (https://openweathermap.org/api)

In [ ]:
cd /content/drive/My\ Drive/st445/lectures2020/Week5

/content/drive/My Drive/st445/lectures2020/Week5


In [ ]:
import json
with open("APItemplate.json", "r") as file:
    API = json.load(file)
api_key=API['OpenWeather']
# api_key = ''
query = 'q='+'London, uk'
res=requests.get('http://api.openweathermap.org/data/2.5/weather?'+query+'&APPID='+api_key)
print(res.json())

{'coord': {'lon': -0.13, 'lat': 51.51}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 285.7, 'feels_like': 280.19, 'temp_min': 284.82, 'temp_max': 287.04, 'pressure': 1002, 'humidity': 77}, 'visibility': 10000, 'wind': {'speed': 7.42, 'deg': 237}, 'clouds': {'all': 100}, 'dt': 1603894716, 'sys': {'type': 3, 'id': 2019646, 'country': 'GB', 'sunrise': 1603867648, 'sunset': 1603903253}, 'timezone': 0, 'id': 2643743, 'name': 'London', 'cod': 200}
